In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import torchvision.transforms.v2 as v2
from torchvision.transforms import transforms
import os 
import sys
import glob
from scipy import fftpack, ndimage
import re
from skimage.morphology import opening, disk
from skimage import img_as_float
from torch.utils.data import DataLoader
from model import BaseUnet, BaseUnet3D
from data_processing_tools import remove_repeating_pattern
import timeit

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: hndrkjs (hndrkjs-danmarks-tekniske-universitet-dtu). Use `wandb login --relogin` to force relogin


In [2]:
one_channel_unet = BaseUnet(num_inputs=1)
one_channel_unet.load_state_dict(torch.load('early_stopping_modelablation_channel_base_unet_1.pth', map_location=torch.device('cpu')))

two_channel_unet = BaseUnet(num_inputs=2)
two_channel_unet.load_state_dict(torch.load('early_stopping_modelablation_channel_base_unet_2.pth', map_location=torch.device('cpu')))

three_channel_unet = BaseUnet(num_inputs=3)
three_channel_unet.load_state_dict(torch.load('early_stopping_modelablation_channel_base_unet_3.pth', map_location=torch.device('cpu')))

four_channel_unet = BaseUnet(num_inputs=4)
four_channel_unet.load_state_dict(torch.load('early_stopping_modelablation_channel_base_unet_4.pth', map_location=torch.device('cpu')))

# five_channel_unet = BaseUnet(num_inputs=5)
# five_channel_unet.load_state_dict(torch.load('early_stopping_modelablation_channel_base_unet_5.pth', map_location=torch.device('cpu')))

# six_channel_unet = BaseUnet(num_inputs=6)
# six_channel_unet.load_state_dict(torch.load('early_stopping_modelablation_channel_base_unet_6.pth', map_location=torch.device('cpu')))

# seven_channel_unet = BaseUnet(num_inputs=7)
# seven_channel_unet.load_state_dict(torch.load('early_stopping_modelablation_channel_base_unet_7.pth', map_location=torch.device('cpu')))

# eight_channel_unet = BaseUnet(num_inputs=8)
# eight_channel_unet.load_state_dict(torch.load('early_stopping_modelablation_channel_base_unet_8.pth', map_location=torch.device('cpu')))

nine_channel_unet = BaseUnet(num_inputs=9)
nine_channel_unet.load_state_dict(torch.load('early_stopping_modelablation_channel_base_unet_9.pth', map_location=torch.device('cpu')))

ten_channel_unet = BaseUnet(num_inputs=10)
ten_channel_unet.load_state_dict(torch.load('early_stopping_modelablation_channel_base_unet_10.pth', map_location=torch.device('cpu')))

eleven_channel_unet = BaseUnet(num_inputs=11)
eleven_channel_unet.load_state_dict(torch.load('early_stopping_modelablation_channel_base_unet_11.pth', map_location=torch.device('cpu')))

base_unet_no_preprocessing = BaseUnet()
base_unet_no_preprocessing.load_state_dict(torch.load('early_stopping_modelUnet_simple_11_nopreprocess.pth', map_location=torch.device('cpu')))

base_unet_no_preprocessing_3d = BaseUnet3D()
base_unet_no_preprocessing_3d.load_state_dict(torch.load('early_stopping_modelUnet3D_simple_11_no_preprocess.pth', map_location=torch.device('cpu')))

base_unet_preprocessing = BaseUnet()
base_unet_preprocessing.load_state_dict(torch.load('early_stopping_modelUnet_simple_11_preprocess.pth', map_location=torch.device('cpu')))

base_unet_preprocessing_3d = BaseUnet3D()
base_unet_preprocessing_3d.load_state_dict(torch.load('early_stopping_modelUnet3D_simple_11_preprocess.pth', map_location=torch.device('cpu')))

one_channel_unet.eval()
two_channel_unet.eval()
three_channel_unet.eval()
four_channel_unet.eval()
five_channel_unet.eval()
six_channel_unet.eval()
seven_channel_unet.eval()
eight_channel_unet.eval()
nine_channel_unet.eval()
ten_channel_unet.eval()
eleven_channel_unet.eval()
base_unet_no_preprocessing.eval()
base_unet_no_preprocessing_3d.eval()
base_unet_preprocessing.eval()
base_unet_preprocessing_3d.eval()

/var/folders/tw/b1bzt3x913368xjdhnk6r2b00000gn/T/ipykernel_3451/2691601514.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  one_channel_unet.load_state_dict(torch.load('e

RuntimeError: Error(s) in loading state_dict for BaseUnet:
	size mismatch for enc_conv0.weight: copying a param with shape torch.Size([32, 11, 3, 3]) from checkpoint, the shape in current model is torch.Size([32, 1, 3, 3]).